## How to download files from your Trello in subfolders?

1. Download csv-file from your board: Menu -> Share -> Export in CSV (access only with Trello Premium :(, json-format not supported here).
2. Copy this code and csv-file in one folder.
3. Add your values in **API_KEY**, **ACCESS_TOKEN**, **csv_file_name** and **folder_name** (see below). You can create API_KEY and ACCESS_TOKEN here: https://trello.com/power-ups/admin
4. Run all.

Link on GitHub: https://github.com/sergevv/download-files-from-trello-boards

## Libraries

In [5]:
import os
import csv
import json
import time
import requests
import numpy as np
import pandas as pd 
from datetime import datetime

## Download files in subfolders

In [6]:
def download_files_from_csv(csv_file_path, folder_name):
    count = 0

    # Create a folder to store files
    files_folder = folder_name
    if not os.path.exists(files_folder):
        os.mkdir(files_folder)

    # Read the CSV file
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            attachment_links = row['Attachment Links'].split(', ')
            card_name = row['Card Name']
            board_name = row['List Name']

            # Skip rows without valid file URLs
            if not attachment_links[0]:
                continue

            # Create folders for the board and card if they don't exist
            board_folder = os.path.join(files_folder, board_name)
            if not os.path.exists(board_folder):
                os.mkdir(board_folder)

            card_folder = os.path.join(board_folder, card_name)
            if not os.path.exists(card_folder):
                os.mkdir(card_folder)                

            try:
                for link in attachment_links:
                    
                    # Add API key and access token to the request headers
                    headers = {'Authorization': f'OAuth oauth_consumer_key="{API_KEY}", oauth_token="{ACCESS_TOKEN}"'}

                    # Download the file and save it in the card's folder
                    response = requests.get(link, headers=headers)
                    if response.status_code == 200:
                        file_extension = os.path.splitext(link)[-1]  # Get the file extension
                        timestamp = datetime.now().strftime("%f")
                        file_filename = f"{card_name}_{timestamp}{file_extension}"
                        file_path = os.path.join(card_folder, file_filename)
                        with open(file_path, 'wb') as file_file:
                            file_file.write(response.content)
                        count += 1
                        print(f"Downloaded {count}/{count_links} for '{card_name}' in '{board_name}': {file_path}")
                    elif response.status_code == 401:
                        print(f"Unauthorized to access attachments for '{card_name}' in '{board_name}'. Please check API key and access token.")
                    else:
                        print(f"Failed to download for '{card_name}' in '{board_name}': {response.status_code}")
            except Exception as e:
                print(f"Error downloading for '{card_name}' in '{board_name}': {e}")

In [ ]:
### --------------------Set your Trello API key, access token, csv file and folder name ------- ###

API_KEY = "YOUR_API_KEY"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"

csv_file_name = "your_csv_file"
folder_name = "your_foldername_to_download_files" # It is not necessary to creat the folder beforehand

### ------------------- Clean csv-file from incorrect symbols ---------- ###

csv_trello_file = pd.read_csv(csv_file_name)
df = pd.DataFrame(csv_trello_file)

df['Card Name'] = df['Card Name'].str.replace("'", "''")  # Replace single quotes with two single quotes
df['Card Name'] = df['Card Name'].str.replace('"', '""')   # Replace double quotes with two double quotes
df['List Name'] = df['List Name'].str.replace("'", "''")  # Replace single quotes with two single quotes
df['List Name'] = df['List Name'].str.replace('"', '""')   # Replace double quotes with two double quotes

def remove_symbols(text):
    return ''.join([c for c in text if c.isalnum() or c.isspace() or (1040 <= ord(c) <= 1103) or (65 <= ord(c) <= 122)])  # Cyrillic and Latin Unicode ranges

# Apply the function to the desired column
df['Card Name'] = df['Card Name'].apply(remove_symbols)
df['List Name'] = df['List Name'].apply(remove_symbols)

# Save cleaned data
df.to_csv('main_manufacturing.csv', index=False)

### ---------------------- Download files ----------------------------- ###

count_links = df['Attachment Count'].sum()

# Record the start time
start_time = time.time()
print("Start Time:", time.ctime(start_time))

if __name__ == "__main__":
    csv_file_path = csv_file_name
    download_files_from_csv(csv_file_path, folder_name)

print('')
# Record the finish time
finish_time = time.time()

# Calculate the elapsed time
elapsed_time = finish_time - start_time

# Print the start and finish times, as well as the elapsed time
print("Start Time:", time.ctime(start_time))
print("Finish Time:", time.ctime(finish_time))
print("Elapsed Time:", elapsed_time, "seconds")
print('Finished!')